## Setup spark and install modules

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

In [4]:
spark = SparkSession.builder.appName("StructuredStreamingBasics").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/04/05 13:13:03 WARN Utils: Your hostname, DELEQ0283302041 resolves to a loopback address: 127.0.1.1; using 172.31.227.62 instead (on interface eth0)
24/04/05 13:13:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/05 13:13:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Practice

First, let's create a source for data stream. For that we can use Netcat to send data over a network connection.
1. In the terminal, type `nc -lk 9999` to run the netcat server on port 9999, and then type in whatever you choose.

*For the curious ones:*
>   * -l flag is used to specify that nc should listen for an incoming connection rather than initiate a connection to a remote host
>   * -k flag forces nc to stay listening for another connection after its current connection is completed.

2. Create DataFrame representing the stream of input lines from connection to localhost:9999

In [5]:
lines = spark.readStream.format("socket").option("host", "localhost").option("port", 9999).load()

24/04/05 13:13:17 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


3. Create another dataFrame that counts words from spliting a line

In [6]:
words = lines.select(explode(split(lines.value, " ")).alias("word"))

In [7]:
wordCounts = words.groupBy("word").count()

4. Start the query that prints the running counts to the console

To kill a Stream, interupt running for cell below

In [8]:
query = wordCounts.writeStream.outputMode("complete").format("console").start()
query.awaitTermination()

24/04/05 13:14:33 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-ca761f6b-4679-4033-8b3b-80677ef745c9. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/05 13:14:33 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+---------+-----+
|     word|count|
+---------+-----+
|  Mariusz|    1|
|Michalina|    1|
|    Hello|    1|
|      and|    1|
+---------+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+---------+-----+
|     word|count|
+---------+-----+
|     hope|    1|
|      day|    1|
|      you|    1|
|  Mariusz|    1|
|    great|    1|
|Michalina|    1|
|     have|    1|
|    Hello|    1|
|      and|    1|
|        I|    1|
|      all|    1|
|        a|    1|
+---------+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+---------+-----+
|     word|count|
+---------+-----+
|     hope|    1|
|     some|    1|
|      day|    1|
|      you|    1|
|  Mariusz|    1|
|   please|    1|
|    great|    1|
|Michalina|    1|
|     have|    1|
|      Now|    1|
|    Hello|    1|
|   actual|    1|
|     work|    1|
|      and|    1|
|       do|    1|
|        I|    1|
|      all|    1|
|        a|    1|
+---------+-----+



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/mnestoro/SPARK/.venv/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/mnestoro/SPARK/.venv/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [9]:
query.stop()